# 음성 데이터 녹음을 위한 노트북

## 요구사항

1. Jupyter Notebook을 사용해 녹음을 진행하기 위해서는 `ffmpeg`을 설치해야 합니다.
2. `pip install torchaudio ipywebrtc notebook pydub`로 필요한 파이썬 라이브러리들을 설치해주세요.
3. `jupyter nbextension enable --py widgetsnbextension`를 실행하여 notebook extension을 실행해주세요.

위 준비가 끝났다면 아래 두 셀을 실행해주세요. 녹음할 준비가 끝났습니다.

In [12]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio
import os
from pydub import AudioSegment

In [17]:
script_file = 'tts1/org/transcripts.txt'
target_scripts = []
for line in open(script_file, 'rt'):
    target_scripts.append(line.strip().split('|')[1])
!mkdir -p org
wav_dir = 'org'
cur_idx = 0
for f in os.listdir(wav_dir):
    if f.endswith('.wav'):
        cur_idx += 1
print("현재 녹음한 문장 의 수:", cur_idx)

현재 녹음한 문장 의 수: 1


# 녹음 시 주의사항

1. 전체 녹음 과정에서 문장을 읽을 때 비슷한 속도와 톤을 유지해주세요. 가령, 어떤 문장은 빠르게 읽고 어떤 문장은 매우 늘려서 읽으면 학습이 잘 되지 않습니다.
2. 녹음 시에 말을 절거나, 실수를 하거나 소음이 섞여 들어가면 다시 녹음해주세요.
3. 녹음 과정 중에 마이크와의 거리를 동일하게 유지해주세요. 


# 녹음 방법

바로 아래 셀을 실행하면 녹음할 문장이 나타납니다. 동그라미 버튼을 누르시고 녹음을 한 뒤, 다시 동그라미 버튼을 누르면 녹음이 끝납니다. 그 뒤 바로 녹음한 음성이 재생됩니다.

In [9]:
if cur_idx == len(target_scripts):
    raise Exception("모든 스크립트의 녹음이 끝난 상태 같습니다.")
    
# from https://medium.com/@harrycblum/record-audio-in-a-jupyter-notebook-da08a88278bb
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
print("문장 번호 :", cur_idx)
print("문장 :", target_scripts[cur_idx])
recorder

Index : 0
Current Script : 여러분은 이곳에서 장미, 백합, 튤립 등 다양한 종류의 꽃들을 보실 수 있습니다.


AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

녹음을 한 뒤 아래 셀을 실행하면 음성 파일이 저장됩니다. 그 뒤 다시 위 셀을 실행하여 녹음을 진행하시면 됩니다.

In [13]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav tmp.wav -y -hide_banner -loglevel panic
AudioSegment.from_file('tmp.wav').export(os.path.join(wav_dir,"%05d.wav"%cur_idx), format="wav")
sig, sr = torchaudio.load(os.path.join(wav_dir, "%05d.wav"%cur_idx))
cur_idx += 1
print(sig.shape)
Audio(data=sig, rate=sr)

torch.Size([1, 69120])
